In [1]:
# importing necessary libs
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2017) 
from keras import regularizers, optimizers
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
import os


Using TensorFlow backend.


In [0]:
# Tunable hyperparameters
BATCH_SIZE = 512 #@param ["512", "256", "128"] {type:"raw"}
MOMENTUM = 0.9 #@param ["0.9", "0.95", "0.975"] {type:"raw"}
WEIGHT_DECAY = 0.000125 #@param ["0.000125", "0.00025", "0.0005"] {type:"raw"}
LEARNING_RATE = 0.4 #@param ["0.4", "0.2", "0.1"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
WARMUP = 5 #@param {type:"slider", min:0, max:24, step:1}

In [3]:
# Getting cifar10 data
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()
num_train, img_rows, img_cols,img_channels =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
#Normalizing data
train_features = train_features.astype('float32')/255
test_features = test_features.astype('float32')/255
# convert class labels to binary class labels
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

In [0]:
#Defining function to measure accuracy
def accuracy(test_x, test_y, model):
    result = model.predict(test_x)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(test_y, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

In [6]:
''' need to impliment this in keras
def build_nn(c=64, weight=0.125):
 
return model
'''

model = Sequential()

#model.add(fw.layers.ConvBN(c, **fw.layers.PYTORCH_CONV_PARAMS))
model.add(Convolution2D(64, 3, 3, input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(fw.layers.ConvResBlk(c*2, res_convs=2,  **fw.layers.PYTORCH_CONV_PARAMS))
model.add(Convolution2D(128, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(128, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(fw.layers.ConvBlk(c*4, **fw.layers.PYTORCH_CONV_PARAMS))
model.add(Convolution2D(256, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(fw.layers.ConvResBlk(c*8, res_convs=2,  **fw.layers.PYTORCH_CONV_PARAMS))
model.add(Convolution2D(512, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(512, 3, 3))
model.add(BatchNormalization())
#model.add(Activation('relu'))

#model.add(tf.keras.layers.GlobalMaxPool2D())
model.add(MaxPooling2D()) # , AveragePooling2D
model.add(GlobalAveragePooling2D())

#model.add(fw.layers.Classifier(n_classes, kernel_initializer=fw.layers.init_pytorch, weight=0.125))
model.add(Dense(num_classes, activation='softmax'))

W0802 17:24:14.770689 140336569206656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(32, 32, 3...)`
  # Remove the CWD from sys.path while we load stuff.
W0802 17:24:14.808055 140336569206656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 17:24:14.814225 140336569206656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 17:24:14.851964 140336569206656 deprecation_wrapper.

In [7]:
#Compiling model 
sgd = optimizers.SGD(momentum=MOMENTUM, nesterov=False)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

W0802 17:24:18.372078 140336569206656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 128)       147584    
__________

In [9]:
#Loading google drive to retrive data and store training weights
from google.colab import drive
drive.mount('/content/gdrive')

#preparing path for dir in drive to save weights
import os
corpus = os.path.join("/content/gdrive/My Drive", "Weights")

# define the checkpoint
filepath=os.path.join(corpus, "13-v0.1weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5")
#"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [10]:
#Getting repo for import OneCycleLR from this link: https://github.com/titu1994/keras-one-cycle
!git clone https://github.com/titu1994/keras-one-cycle.git
import sys
sys.path.append("/content/keras-one-cycle/")

Cloning into 'keras-one-cycle'...
remote: Enumerating objects: 208, done.
remote: Total 208 (delta 0), reused 0 (delta 0), pack-reused 208
Receiving objects: 100% (208/208), 3.18 MiB | 16.45 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [11]:
#Finding LR
from clr import LRFinder

lr_callback = LRFinder(num_samples=num_train, batch_size=BATCH_SIZE,
                       minimum_lr=0.001, maximum_lr=0.1,
                       # validation_data=(X_val, Y_val),
                       lr_scale='exp', save_dir=filepath)

# Ensure that number of epochs = 1 when calling fit()
model.fit(train_features, train_labels, epochs=1, batch_size=BATCH_SIZE, callbacks=[lr_callback])

W0802 17:24:50.555277 140336569206656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
49664/50000 [============================>.] - ETA: 0s - loss: 1.6209 - acc: 0.4220 - LRFinder: lr = 0.10000000 
 - LRFinder: lr = 0.10486210 
50000/50000 [==============================] - 48s 968us/step - loss: 1.6205 - acc: 0.4222
	LR Finder : Saved the losses and learning rate values in path : {/content/gdrive/My Drive/Weights/13-v0.1weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5}


In [0]:
#Training with OneCycleLR
from clr import OneCycleLR

lr_manager = OneCycleLR(num_samples=num_train, num_epoch, batch_size=BATCH_SIZE, max_lr,
                        end_percentage=0.1, scale_percentage=None,
                        maximum_momentum=0.95, minimum_momentum=0.85)
                        
model.fittrain_features, train_labels, epochs=EPOCHS, batch_size==BATCH_SIZE, callbacks=[model_checkpoint, lr_manager])

Data augmentation and input pipeline. In DavidNet, training images go through the standard Cifar10 transformations, that is, pad 4 pixels to 40×40, crop back to 32×32, and randomly flip left and right. In addition, it applies the popular Cutout augmentation as a regularization measure, which alleviates overfitting. Cutout is a bit tricky to implement in Tensorflow.

In [0]:
#Applying data augmentation


In [0]:
#Training Model

In [0]:
#Testing Model